**Calculate the Maximum Pressure, Area Size, Centroid_0, Centroid_1, and Centroid_2 for Both MR-based and CT-based Simulations**
----
### Context
This notebook shows an example that calculates the outcomes using MR-based and CT-based Simulations for the 54 EEG locations. It is modified for the data with SDR = 0.55. This code can also be adapted for different datasets.


In [ ]:
import numpy as np
from skimage.measure import label, regionprops, regionprops_table
import nibabel
import pandas as pd
import os
import matplotlib.pyplot as plt

## Additions to path

In [ ]:
inputcsv='SDR_0p55_eeg_normals.csv'
SELECTED_FOR_FUS = pd.read_csv('SELECTED_EEG.csv',index_col='Name')['Include'].to_dict()

In [ ]:
BasePath='ZenodoFinal/SDR_0p55/m2m_SDR_0p55'    
T1W = 'ZenodoFinal/SDR_0p55/T1W.nii.gz'
simbnibs_path=BasePath
Frequency=500e3
PPW =6
TxSystem='Single'
Aperture = 50e-3
FocalLength = 50e-3
zLengthBeyonFocalPointWhenNarrow = 80e-3

In [ ]:
# Initialize lists to store data
all_data_max = []

# Loop through selected IDs
for ID in SELECTED_FOR_FUS:
    if SELECTED_FOR_FUS[ID] == 'YES':
        print('*' * 50)
        print(f"Processing ID: {ID}")
        print('*' * 50)

        # File paths
        Charm_path = os.path.join('MRI_vs_CT/SDR_0p55/',
                                  f"{ID}_Single_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_FullElasticSolution_Sub_NORM.nii.gz")
        CT_path = os.path.join('MRI_vs_CT/SDR_0p55/',
                               f"CT_{ID}_Single_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_FullElasticSolution_Sub_NORM.nii.gz")

        # Ensure both files exist
        assert os.path.isfile(Charm_path) and os.path.isfile(CT_path)

        # Load data
        Charm_data = nib.load(Charm_path).get_fdata()
        CT_data = nib.load(CT_path).get_fdata()

        # Label regions and compute properties
        Charm_label_img = label(Charm_data >= 0.5)
        CT_label_img = label(CT_data >= 0.5)

        Charm_props = regionprops_table(Charm_label_img, properties=('label', 'area', 'centroid'))
        CT_props = regionprops_table(CT_label_img, properties=('label', 'area', 'centroid'))

        # Find the largest area label
        max_area_index_ch = np.argmax(Charm_props['area'])
        max_area_index_ct = np.argmax(CT_props['area'])

        # Load unnormalized data for pressure calculation
        Charm_path_unnorm = os.path.join('MRI_vs_CT/SDR_0p55/',
                                         f"{ID}_Single_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_FullElasticSolution_Sub.nii.gz")
        CT_path_unnorm = os.path.join('MRI_vs_CT/SDR_0p55/',
                                      f"CT_{ID}_Single_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc{FocalLength*1e3:.1f}_Diam{Aperture*1e3:.1f}_FullElasticSolution_Sub.nii.gz")

        Charm_data_unnorm = nib.load(Charm_path_unnorm).get_fdata()
        CT_data_unnorm = nib.load(CT_path_unnorm).get_fdata()

        # Calculate maximum pressures using centroids
        Charm_centroid = tuple(np.round([Charm_props[f'centroid-{i}'][max_area_index_ch] for i in range(3)]).astype(int))
        CT_centroid = tuple(np.round([CT_props[f'centroid-{i}'][max_area_index_ct] for i in range(3)]).astype(int))

        MaxCharmPressure = Charm_data_unnorm[Charm_centroid]
        MaxCTPressure = CT_data_unnorm[CT_centroid]

        # Store results for the largest label
        data_max = {
            'ID': ID,
            'Max Charm Pressure': MaxCharmPressure,
            'Max CT Pressure': MaxCTPressure,
            'Area_ch': Charm_props['area'][max_area_index_ch],
            'Area_ct': CT_props['area'][max_area_index_ct],
            'centroid-0_charm': Charm_centroid[0],
            'centroid-1_charm': Charm_centroid[1],
            'centroid-2_charm': Charm_centroid[2],
            'centroid-0_ct': CT_centroid[0],
            'centroid-1_ct': CT_centroid[1],
            'centroid-2_ct': CT_centroid[2]
        }

        all_data_max.append(data_max)

# Convert to DataFrame and save
df_all_max = pd.DataFrame(all_data_max)
output_file = 'output.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    df_all_max.to_excel(writer, sheet_name='Max Area', index=False)

print(f"Data saved to {output_file}")